**Dependencies**

In [3]:
import string,re
from unicodedata import normalize
from numpy import array,argmax
from pickle import load,dump
from numpy.random import rand,shuffle

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential,load_model
from keras.layers import LSTM,Dense,Embedding,RepeatVector,TimeDistributed
from nltk.translate.bleu_score import SmoothingFunction,corpus_bleu
smoothie = SmoothingFunction().method4

**Loading the file and reading the content of the file**

In [5]:
# load file into memory
def load_file(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

**Splitting the Sentences into Words**

In [6]:
# split a loaded document into sentences
def splitting_sentence(doc):
	sentences = doc.strip().split('\n')
	pairs = [sentence.split('\t') for sentence in  sentences]
	return pairs

**Cleaning the pairs**

In [7]:
# cleaning a list of sentences and creating pairs

def clean_pairs(sentences):
	cleaned = list()
 
	# preparing regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))

	# preparing translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)

  # iterating over each pair
	for pair in sentences:
		clean_pair = list()
  
		for sentence in pair:
			# normalizing unicode characters
			sentence = normalize('NFD', sentence).encode('ascii', 'ignore')
			sentence = sentence.decode('UTF-8')
			# tokenizing on white space
			sentence = sentence.split()
			# converting to lowercase
			sentence = [word.lower() for word in sentence]
			# removing punctuation from each token
			sentence = [word.translate(table) for word in sentence]
			# removing non-printable chars form each token
			sentence = [re_print.sub('', w) for w in sentence]
			# removing tokens with numbers in them
			sentence = [word for word in sentence if word.isalpha()]
			# storing as string
			clean_pair.append(' '.join(sentence))
		cleaned.append(clean_pair)
	return array(cleaned)

**Saving the Cleaned data**

In [8]:
def saving_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print(filename,': Saved')

**Saving data in .pkl format**

In [9]:
# load dataset

filename = 'fra.txt'
doc = load_file(filename)

# split into english-french pairs
pairs = splitting_sentence(doc)

# clean sentences
clean_pairs = clean_pairs(pairs)

# save clean pairs to file
saving_clean_data(clean_pairs, 'english-french.pkl')

print('English','-->',"French")
# spot check
for i in range(25):
	print(clean_pairs[i,0],'-->',clean_pairs[i,1])

**Loading the cleaned data**

In [10]:
# load a clean dataset
def loading_cleaned_data(filename):
	return load(open(filename, 'rb'))

In [11]:
# load dataset
data = loading_cleaned_data('english-french.pkl')
print(data.shape) 

**Scaling of data** 

**Size**

1.Dataset - 30000

2.Training - 25000

3.Testing - 5000   



In [12]:
# reducing dataset size (scaling) 

new_data_size = 30000
dataset = data[:new_data_size, :]

# randomly shuffling the dataset to get proper training and testing data
shuffle(dataset)

# splitting into training and testing (90%-10%)
train, test = dataset[:25000], dataset[25000:]

# saving the cleaned data,train data and test data 
saving_clean_data(dataset, 'english-french-both.pkl')
saving_clean_data(train, 'english-french-train.pkl')
saving_clean_data(test, 'english-french-test.pkl')

In [13]:
# loading datasets and saving it into variables
dataset = loading_cleaned_data('english-french-both.pkl')
train = loading_cleaned_data('english-french-train.pkl')
test = loading_cleaned_data('english-french-test.pkl')

**Creating a tokenizer for the lines and finding the maximum length phrase**

In [14]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

**Size of English & French vocabulary and their max phrase length**

In [15]:
# preparing the english tokenizer

eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])

print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))

# preparing the french tokenizer

fra_tokenizer = create_tokenizer(dataset[:, 1])
fra_vocab_size = len(fra_tokenizer.word_index) + 1
fra_length = max_length(dataset[:, 1])
print('French Vocabulary Size: %d' % fra_vocab_size)
print('French Max Length: %d' % (fra_length))


**Encoding to integers and padding to the maximum phrase length**

In [16]:
# Input and Output sequence must be encoded to integers and padded to the maximum phrase length
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	x = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	x = pad_sequences(x, maxlen=length, padding='post')
	return x

# One hot encoding to max phrase length
def one_hot_encoding(sequences, vocab_size):
	y_1 = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		y_1.append(encoded)
	y = array(y_1)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

**Training and Testing Data**

In [17]:
# preparing training data
trainX = encode_sequences(fra_tokenizer, fra_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = one_hot_encoding(trainY, eng_vocab_size)

# prepare testing data
testX = encode_sequences(fra_tokenizer, fra_length, test[:, 1])
testY = encode_sequences(eng_tokenizer,eng_length, test[:, 0])
testY = one_hot_encoding(testY, eng_vocab_size)

In [18]:
print('training size:',trainX.shape,trainY.shape)
print('testing size:',testX.shape,testY.shape)

**Building the model**

In [19]:
def model_building(source_vocab, target_vocab, source_len, target_len, units):
	model = Sequential()
	model.add(Embedding(source_vocab, units, input_length=source_len, mask_zero=True))
	model.add(LSTM(units))
	model.add(RepeatVector(target_len))
	model.add(LSTM(units, return_sequences=True))
	model.add(TimeDistributed(Dense(target_vocab, activation='softmax')))
	return model

**Defining and Compiling the model**

In [20]:
model = model_building(fra_vocab_size, eng_vocab_size, fra_length, eng_length, 512)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])

**Model Summary**

In [21]:
print(model.summary())

In [22]:
# Stop model if accuracy of the model doesn't changes by more than 0.01 
# Patience = 5 : After each 5 epochs if no improvement is there then training will be stopped.
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_acc',patience = 15, min_delta = 0.05)

**Fitting the model**

1.Epochs = 50

2.Batch_size = 25

In [23]:
# fit model
model.fit(trainX, trainY, epochs= 50, batch_size=25, validation_data=(testX, testY), verbose=2, callbacks = [es])

**Evaluating model and calculating BLEU Score**

Evaluation involves two steps: 

1.Generating a translated output sequence, and 

2.then repeating this process for many input examples and summarizing the skill of the model across multiple cases.

In [24]:
# mapping integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [25]:
# generating target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

In [26]:

# evaluating the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
  
  # Creating empty lists for actual phrases(French) and predicted phrases(English) 
  actual,predicted = list(),list()
  a,b,c = list(),list(),list()
  for i,source in enumerate(sources):

    # reshaping to the required size
    source = source.reshape((1, source.shape[0]))

    # predicting for the english tokenizer
    translation = predict_sequence(model, eng_tokenizer, source)
    # raw_dataset = raw_dataset[i].split(' ') 
    # print(raw_dataset[i][1])

    raw_src,raw_target = raw_dataset[i][1],raw_dataset[i][0]
    
    # First 10 Predictions
    if i <= 299:
        print(f"{i + 1}) SOURCE = {raw_src}")
        print(f"   TARGET = {raw_target}")
        print(f"   PREDCITED = {translation}")
        print()

#     actual.append([raw_target.split()])
#     predicted.append(translation.split())
  
 #   calculating BLEU score
#   print('-------------------------------------------')
#   print('BLEU Score :')
#   print('BLEU score-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0),smoothing_function=smoothie,auto_reweigh=False))
#   print('BLEU score-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0),smoothing_function=smoothie,auto_reweigh=False))
#   print('BLEU score-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0),smoothing_function=smoothie,auto_reweigh=False))
#   print('BLEU score-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie,auto_reweigh=False))

**Evaluating Model on training data**

**Evaluating Model on testing data**

In [27]:
evaluate_model(model,eng_tokenizer,trainX[:20],train[:20])

In [28]:
evaluate_model(model, eng_tokenizer, testX[:20], test[:20])

In [29]:
# Saving the model for Future Inferences

model_json = model.to_json()  # Saving the Architecture Of the Model in a JSON File
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")  # Saving the Weight Parameters Of the Model

**Performing Predictions on User Input**

In [30]:
def frnch_eng(text):
    sentc = encode_sequences(fra_tokenizer, fra_length, [text])
    trans = predict_sequence(model, eng_tokenizer, sentc)
    return trans

In [32]:
frnch_eng("bois davantage deau")